<a href="https://colab.research.google.com/github/promptmule4real/demo/blob/main/promptmule_reporting_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PromptMule API Demo Suite

Welcome to the PromptMule API Demo Suite! This code offers you a comprehensive way to test your interactions with the PromptMule API by generating various prompts, analyzing the semantic scores of responses, and compiling detailed reports on the tokens used and saved through caching. Let's walk through what this code does and how you can make the most of it.

#### What is this Code?

This script facilitates your interaction with the PromptMule API, a platform that offers optimized API requests to different AI models. By using this demo suite, you can:
- Authenticate and get a token for API interactions.
- Generate or fetch your user-specific API key.
- Send a series of prompt requests to the API, with varying semantic similarities and max response counts.
- Track and print the detailed logs of each API response (if verbose mode is enabled).
- Compile and display a structured report of the prompt responses and cache savings.

#### How to Use:

1. **Start the Script**: Run the script in a Python environment.
2. **Enter User Details**: You'll be prompted to enter your username, password, and the name of your application.
3. **Choose Verbose Mode**: If you'd like to see detailed logs for each API call/response, choose "yes" for verbose mode. If you prefer a silent mode with minimal output, select "no".
4. **Provide Cost-saving Strategy**: Input the name of your cost-saving strategy when prompted. This will be used to generate prompts for testing.
5. **Review the Reports**: After all the prompts have been sent and responses received, you'll get:
    - A detailed report of the prompt responses.
    - A summary of your usage.
    - A report on the savings achieved using caching.
6. **Profile Deletion (Optional)**: At the end, you'll be given the option to delete your user profile. Only choose this if you're sure, as this action cannot be undone.

#### Additional Information:
For detailed documentation, further guidance, or any other queries related to the PromptMule API, please visit [PromptMule](https://www.promptmule.com/).

Happy testing!

In [ ]:
import pandas as pd
import requests
import json
import random
import time

# Constants for API interactions
ENDPOINT = 'https://api.promptmule.com/'
LOGIN = 'login'
KEY_GEN = 'api-keys'
PROMPT = 'prompt'
LIST_API_KEYS = 'api-keys/'
DELETE = 'profile/'

# User-specific details
USERNAME = input("Please provide your username: ")
PASSWORD = input("Please provide your password: ")
APPNAME = input("Please provide your app name: ")

verbose_mode = input("Do you want verbose mode enabled (detailed logs)? (yes/no): ").lower() == "yes"

headers = {'Content-Type': 'application/json'}

# Report Table
report_df = pd.DataFrame(columns=['Prompts Sent', 'Tokens Sent', 'Responses Received', 'Tokens Received', 'Cached Responses', 'Tokens Saved by Cache'])

# Trackers for the report data
prompts_sent = 0
tokens_sent = 0
responses_received = 0
tokens_received = 0
cached_responses = 0
tokens_saved_by_cache = 0

# The rest of the function definitions remain unchanged

def send_prompt_requests(cost_saving_strategy, semantic_similarity, max_response_quantity):
    global prompts_sent, tokens_sent, responses_received, tokens_received, cached_responses, tokens_saved_by_cache

    for variation in range(1, int(prompt_variations * 0.9) + 1):
        prompt_text = generate_prompt(cost_saving_strategy, variation)
        prompts_sent += 1
        tokens_sent += len(prompt_text.split())

        api_call_body = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": prompt_text}],
            "max_tokens": "50",
            "temperature": "0.99",
            "user": USERNAME,
            "api": "openai",
            "semantic": str(semantic_similarity),
            "sem_num": str(max_response_quantity)
        }

        start_time = time.time()
        response = requests.post(ENDPOINT + PROMPT, headers={'x-api-key': api_key, 'Content-Type': 'application/json'}, json=api_call_body)
        latency = time.time() - start_time

        if response.status_code == 200:
            responses_received += 1

            choices_data = track_semantic_scores(response.json())
            tokens_received += sum([len(choice[2].split()) for choice in choices_data])

            # Check if a response is cached
            for choice in choices_data:
                if "cached" in choice and choice["cached"] == True:
                    cached_responses += 1
                    tokens_saved_by_cache += len(choice[2].split())

            if verbose_mode:
                print("\nPrompt response:")
                print(json.dumps(response.json(), indent=4))
                print(f"Latency: {latency} seconds")
                print("\nTop Implementation Tip:", choices_data[0][2])
                print("\nOther Tips:", *[choice[2] for choice in choices_data[1:]], sep="\n")
        else:
            print(f"\nError with prompt variation {variation}. Status code: {response.status_code}")
            print(json.dumps(response.json(), indent=4))

# Main Execution
# 1. Log in to the service and update headers with the obtained token
token = login_to_promptmule()
headers["Authorization"] = f"Bearer {token}"
print("Login successful! Token acquired.")

# 2. Generate or retrieve the user's API key
api_key = generate_or_fetch_api_key()
print(f"API Key: {api_key}")

# 3. Gather user input for prompt testing
cost_saving_strategy = input("What do you call your cost-savings strategy while implementing AI applications?: ")

# 4. Define constants for prompt variations and requests
prompt_variations = 25
requests_per_variation = 5
df = pd.DataFrame(columns=['Prompt ID', 'Semantic Similarity', 'Score', 'Choice Content', 'Choice Index'])

# 5. Send prompt requests with varying semantic similarities and max response counts
semantic_nums = [1.0, 0.75, 0.0]
max_responses = list(range(1, 11))  # From 1 to 10
for sem_num in semantic_nums:
    send_prompt_requests(cost_saving_strategy, sem_num, random.choice(max_responses))

# 6. Display the results in a structured report
print("\nReport:")
print(df.to_string(index=False))

def fetch_data(endpoint, headers, success_message, fail_message):
    """Make a GET request and print the results."""
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        print(f"\n{success_message}:")
        print(json.dumps(response.json(), indent=4))
    else:
        print(f"\n{fail_message} {response.status_code}:")
        print(json.dumps(response.json(), indent=4))


# 7. Fetch and display user data summary
fetch_data(ENDPOINT + LIST_API_KEYS, headers, "List of API keys", "Failed to list API keys")
fetch_data(ENDPOINT + 'usage', {"Authorization": f"Bearer {token}", "x-api-key": api_key}, "Usage Summary", "Failed to fetch usage summary")
fetch_data(ENDPOINT + 'usage/daily-stats', {"Authorization": f"Bearer {token}", "x-api-key": api_key}, "Daily Usage Stats", "Failed to fetch daily usage stats")

print("\nReport Summary:")
print(f"Username: {USERNAME}")
print(f"API Key: {api_key}")
print(f"Number of Cost-saving Strategy Variations Sent: {prompt_variations}")
print(f"Number of Requests per Strategy Variation: {requests_per_variation}")

# 8. Optionally allow user to delete their profile
if input("\nDo you really want to delete your user profile? (yes/no): ").lower() == 'yes':
    delete_data(ENDPOINT + DELETE, headers)

# Append data to report table
report_df.loc[len(report_df)] = [prompts_sent, tokens_sent, responses_received, tokens_received, cached_responses, tokens_saved_by_cache]

# Display the report
print("\nPromptmule Cache Report:")
print(report_df.to_string(index=False))

